## Six Step Dance via Langchain
1. Taking into Account Company Industry (Growth / Value)
    1. Growth / Value
    2. Sector outperforming market or not
    3. Company outperforming sector
    4. PE vs Industry
2. Determining Company Financial Stability
    1. Debt To Equity
    2. Current Ratio
    3. Interest Margin Cover
3. Looking at Historical Earning and Growth
    1. Sales Growth
    2. EPS Growth
    3. Return on Invested Capital
    4. Profit Margin
    5. Cashflow Growth
    6. Equity Growth
    7. Return on Equity
4. Understanding Earnings and Sales Expectations
    1. Analyst Estimates
    2. PEG
5. Checking out competition
    1. P/E
    2. P/S
    3.MOAT
6. Estimating Company Values
    1. Insider Buying
    2. Senate TRading
    3. 13f

## Buffett Criteria
### Gross Margin Indicates profitability > 40% GrossProfit/Revenue GROSS PM close to 40 for recent years
### SG&A Margin efficiency in managing overhead costs < 30% SG&A Expenses / Gross Profit
### R&D Margin r&d spending relative to profitability < 30% R&D Expenses / Gross Profit
### Interest Expense Margin reliance on debt and financial health < 15 % Interest Expense / Operating Income
### Income Tax Margin Gauges tax efficiency ~ 20% Income Tax / Pre-Tax Income
### Profit Margin Shows overall profitabilty relative to revenue > 20% Net Income / Revenue
### EPS Growth Indicates growth and consistency in earnings >0 and growing Net Income / Shares Outstanding




### Attempting to write a Small Agent

In [2]:
from openbb import obb
import os
from langchain_openai import ChatOpenAI
from langchain.agents import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [3]:
## Testing some equity screeners
desc_filters = {
        'Market Cap.': '+Small (over $300mln)',
        'Average Volume': 'Over 200K',
        #'Relative Volume': 'Over 1',
    }
fund_filters = {
    'Return on Equity': 'Positive (>0%)',
    'InstitutionalOwnership': 'Under 60%',
    'Current Ratio' :  'Over 1.5',
    'Debt/Equity'   : 'Over 0.3'
}

desc_filters.update(fund_filters)

obb.equity.screener(provider='finviz', industry='semiconductors', 
                    filters_dict=desc_filters
                    )


OBBject

id: 067fc328-71a4-7db9-8000-c5f4a3bdbea7
results: [{'symbol': 'HIMX', 'name': 'Himax Technologies ADR', 'earnings_date': Non...
provider: finviz
warnings: None
chart: None
extra: {'metadata': {'arguments': {'provider_choices': {'provider': 'finviz'}, 'sta...

In [6]:
filter_dict = {
                   'InstitutionalOwnership': 'Under 60%',
                   'Current Ratio' :  'Over 1.5',
                   'Debt/Equity'   : 'Over 0.5',
                   'Return on Assets' : 'Over +10%',#
                   'Average Volume' : 'Over 100K',
                   'Price': 'Over $10',#
                   'InstitutionalOwnership': 'Under 60%',
                   'EPS growthpast 5 years' : 'Positive (>0%)',#
                   'P/E' : 'Under 15',#
                   'P/B': 'Under 2', #
                   }
obb.equity.screener(provider='finviz', industry='beverages_brewers', 
                    filters_dict=desc_filters
                    )

OBBject

id: 067fc34b-393c-7e34-8000-981cad8d2561
results: [{'symbol': 'CCU', 'name': 'Compania Cervecerias Unidas S.A. ADR', 'earnin...
provider: finviz
warnings: None
chart: None
extra: {'metadata': {'arguments': {'provider_choices': {'provider': 'finviz'}, 'sta...

In [ ]:
@tool
def get_industry_performance() -> list:
    """ Return performance by industry for last week, last month, last quarter, last half year and last year"""
    return obb.equity.compare.groups(group='industry', metric='performance').to_llm()

@tool
def get_strong_buy_for_sector(sector : str) -> list :
    """ Return the strong buy recommendation for a given sector"""
    new_sector = '_'.join(sector.lower().split())
    data = obb.equity.screener(provider='finviz', sector=new_sector, recommendation='buy')
    return data.to_llm()

@tool
def get_strong_buy_for_industry(industry : str) -> list :
    """ Return the strong buy recommendation for a given industry"""
    data = obb.equity.screener(provider='finviz', industry=industry, recommendation='buy')
    return data.to_llm()

@tool
def get_best_stock_performers_for_sector(sector:str) -> list :
    """ Return the best  5 stock performers for last week and last month for a given sector"""
    data = obb.equity.screener(provider='finviz', filters_dict={'Sector' : sector, 'Performance' : 'Week Up', 'Performance 2' : 'Month Up'}, limit=5)
    return data.to_llm()

@tool
def get_best_stock_performers_for_industry(industry:str) -> list :
    """ Return the best  5 stock performers for last week and last month for an industry"""
    data = obb.equity.screener(provider='finviz', filters_dict={'Industry' : industry, 'Performance' : 'Week Up', 'Performance 2' : 'Month Up'}, limit=3)
    return data.to_llm()

def get_swing_trade_companies(industry:str) -> list:
    """ Return swing trade companies  by fundamentals. low debt to equity, current ratio  >=1.5, institutional investor < 60, roe >=20%"""
    pass

@tool
def get_valuation_for_sectors(input:str) -> list:
    """ Return valuation metrics for the sector provided as input"""
    data = obb.equity.compare.groups(group='sector', metric='valuation', provider='finviz').to_df()
    
    filtered =  data[data.name == input]
    return filtered.to_json(
            orient="records",
            date_format="iso",
            date_unit="s",
        )

@tool
def get_valuation_for_industries(input:str) -> list:
    """ Return valuation meetrics for the industry provided as input"""
    data =  obb.equity.compare.groups(group='industry', metric='valuation', provider='finviz').to_df()
    filtered =  data[data.name == input]
    return filtered.to_json(
            orient="records",
            date_format="iso",
            date_unit="s",
        )

@tool
def get_valuation_for_company(ticker:str) -> list:
    """ Return Valuation metrics for a company"""
    # requires obb login
    obb.account.login(pat=os.environ['PAT_KEY'])
    return obb.equity.fundamental.ratios(symbol='AAPL', provider='fmp', limit=1).to_llm()

@tool
def get_income_statement(ticker:str) -> list:
    """ Return the last 3  annual income statement for  a given ticker"""
    return obb.equity.fundamental.income(symbol=ticker, limit=3, provider='yfinance').to_llm()



get_strong_buy_for_sector.invoke('Consumer Cyclical')
#get_valuation_for_sectors.invoke('Utilities')
#get_valuation_for_company.invoke('AAPL')
#get_valuation_for_industries.invoke('Medical Distribution')
#get_best_stock_performers_for_industry('Utilities')
get_industry_performance.invoke(None)

'[{"name":"Broadcasting","performance_1d":-0.0249,"performance_1w":-0.1569,"performance_1m":-0.3428,"performance_3m":-0.3664,"performance_6m":-0.3119,"performance_1y":-0.2981,"performance_ytd":-0.3018,"analyst_recommendation":1.93,"volume":25940000,"volume_average":37970000,"volume_relative":0.68},{"name":"Mortgage Finance","performance_1d":0.0162,"performance_1w":-0.1422,"performance_1m":-0.0245,"performance_3m":0.0299,"performance_6m":-0.1168,"performance_1y":0.0677,"performance_ytd":-0.035,"analyst_recommendation":2.47,"volume":27260000,"volume_average":16370000,"volume_relative":1.66},{"name":"REIT - Mortgage","performance_1d":0.0082,"performance_1w":-0.0669,"performance_1m":-0.157,"performance_3m":-0.085,"performance_6m":-0.158,"performance_1y":-0.1172,"performance_ytd":-0.1071,"analyst_recommendation":1.93,"volume":133840000,"volume_average":79200000,"volume_relative":1.69},{"name":"Real Estate - Development","performance_1d":0.0044,"performance_1w":-0.0596,"performance_1m":-0.07

In [7]:
### instructions
instruction = """You are a helpful chatbot that can interact with an SQL database
for a computer store. You will take the users questions and turn them into SQL
queries using the tools available. Once you have the information you need, you will
answer the user's question using the data returned.

Use list_tables to see what tables are present, describe_table to understand thes
schema, and execute_query to issue an SQL SELECT query."""

In [8]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """ You are very powerful stock financial analys, but don't know current events,
                so you will use the tools available to answer any user question""",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [9]:
tools = [get_industry_performance, get_income_statement, get_best_stock_performers_for_industry, get_valuation_for_industries]
llm_with_tools = llm.bind_tools(tools)

In [8]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain_core.output_parsers import StrOutputParser, CommaSeparatedListOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

### Chat Memory

In [3]:
from langchain_core.prompts import MessagesPlaceholder
from langchain.memory import ConversationTokenBufferMemory
#from langchain_core.messages import count_tokens_approximately
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain_core.output_parsers import StrOutputParser, CommaSeparatedListOutputParser
from langchain.agents import AgentExecutor
from langchain_core.messages import AIMessage, HumanMessage




MEMORY_KEY = "chat_history"
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """ You are very powerful stock financial analys, but don't know current events. 
                You  will take the user questions and answer using the tools available.
                Once you have the information you  need, you will answer user's questions using the data returned""",
        ),
        MessagesPlaceholder(variable_name=MEMORY_KEY),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [11]:
tools = [get_industry_performance, get_best_stock_performers_for_industry, get_valuation_for_industries]
llm_with_tools = llm.bind_tools(tools)

In [12]:
chat_history = []
chat_history.append(HumanMessage(content="Your question here"))
chat_history.append(AIMessage(content="AI response here"))
memory = ConversationTokenBufferMemory(
    llm=llm,  # Required for token counting
    max_token_limit=14000,  # Leave buffer for functions + responses
    memory_key="chat_history",  # Must match your prompt's key
    return_messages=True
)

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: memory.load_memory_variables(x)["chat_history"],
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

/tmp/ipykernel_10047/1189149751.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationTokenBufferMemory(


In [13]:
input1 = '''
First, Find the industry that has shown a constant postive performance across quarter, month and week.
Second, extract the valuation metrics for this industry.
Lastly, summarize your finding using not more than 200 words:

'''
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)
print(result['output'])



> Entering new AgentExecutor chain...

Invoking: `get_industry_performance` with `{}`


[{"name":"Broadcasting","performance_1d":-0.1013,"performance_1w":-0.2586,"performance_1m":-0.3293,"performance_3m":-0.3764,"performance_6m":-0.2845,"performance_1y":-0.3075,"performance_ytd":-0.284,"analyst_recommendation":1.92,"volume":26660000,"volume_average":38160000,"volume_relative":0.7},{"name":"Oil & Gas Drilling","performance_1d":-0.1231,"performance_1w":-0.1634,"performance_1m":-0.2423,"performance_3m":-0.4147,"performance_6m":-0.4485,"performance_1y":-0.5724,"performance_ytd":-0.4002,"analyst_recommendation":1.89,"volume":78360000,"volume_average":61380000,"volume_relative":1.28},{"name":"Mortgage Finance","performance_1d":-0.0525,"performance_1w":-0.1473,"performance_1m":-0.052,"performance_3m":-0.0183,"performance_6m":-0.1304,"performance_1y":0.0049,"performance_ytd":-0.0503,"analyst_recommendation":2.47,"volume":28970000,"volume_average":16200000,"volume_relative":1.79},{"name":"Oil

### Let's try to do another approach, We start with the industry, find the ones that are performing better than the industry and narrow down by criteria

In [22]:
input1 = '''
First, analyze the available data to find a industry that has consistently shown positive performance across quarterly, monthly, and weekly timeframes. 
Second, once you have identified the industry, extract its relevant valuation metrics (e.g., P/E, P/B, EV/EBITDA). 
Third, analyze the individual companies within the identified industry to determine the top performers based on relevant metrics like stock price appreciation and revenue growth. 
Fourth, extract the income statement for  the top performer. 
Finally, using income statement data, compute the revenue growth for the past 3 years.'''
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
print(result['output'])



> Entering new AgentExecutor chain...

Invoking: `get_valuation_for_sectors` with `{'input': 'all'}`


[]
Invoking: `get_valuation_for_industries` with `{'input': 'all'}`
responded: I couldn't retrieve the valuation metrics for all industries at once. Let me try to get the valuation metrics for each industry individually to identify the industry that has consistently shown positive performance across quarterly, monthly, and weekly timeframes.

[]
Invoking: `get_valuation_for_industries` with `{'input': 'quarterly'}`
responded: It seems I am unable to retrieve the valuation metrics for all industries at once. Let me try to get the valuation metrics for each industry individually to identify the industry that has consistently shown positive performance across quarterly, monthly, and weekly timeframes.

[]
Invoking: `get_valuation_for_industries` with `{'input': 'monthly'}`
responded: It seems I am unable to retrieve the valuation metrics for all industries at once. Let me try to get th

OpenBBError: 
[Unexpected Error] -> ValueError -> Invalid filter option 'technology'. Possible filter options: ['Any', 'Basic Materials', 'Communication Services', 'Consumer Cyclical', 'Consumer Defensive', 'Energy', 'Financial', 'Healthcare', 'Industrials', 'Real Estate', 'Technology', 'Utilities']

In [ ]:
input1 = "Using the recently found strong buys for Utilities , find their valuation metrics and compare it with the Utilities sector valuation metrics"
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
print(result['output'])

In [ ]:
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)
agent_executor.invoke({"input": "Which stock are recommented for this sector best performing sector", "chat_history": chat_history})